# 장애인콜시스템 가변수 생성
[파이썬 판다스 요일 추출 방법](https://jimmy-ai.tistory.com/331)  
[파이썬 공휴일 리스트 생성](https://velog.io/@vanang7/Python%EC%9C%BC%EB%A1%9C-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%EB%A6%AC%EC%8A%A4%ED%8A%B8%EB%A5%BC-%EB%A7%8C%EB%93%A4%EC%9E%90)  
[공휴일 조회하여 출력하기 (공공데이터포털)](https://sjblog1.tistory.com/59)  
[파이썬 판다스 datetime 타입 시간/주/일 더하기](https://pydole.tistory.com/entry/pandas-datetime-%ED%83%80%EC%9E%85-%EC%8B%9C%EA%B0%84%EC%A3%BC%EC%9D%BC-%EB%8D%94%ED%95%98%EA%B8%B0)

**데이터 불러올 때 구글 드라이브 마운트 실행 필수!**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import holidays

## 데이터 불러오기

In [2]:
# 데이터 불러오기(구글 드라이브 마운트 실행 필수!)
df = pd.read_csv('/content/drive/MyDrive/03. 장애인 콜택시 논문/데이터/장애인콜택시 승하차 내역/장애인콜시스템(21.01.01~21.12.31)정제ver.csv', encoding='cp949')
df = df.drop('Unnamed: 0', axis=1)
df.head()

,차량고유번호,차량타입,예정일시,배차일시,승차일시,출발지구군,출발지상세,목적지구군,목적지상세,대기시간(m),대기시간(s)
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,강남구,수서동,강남구,개포4동,22.10,1326
1,7846,중형 승합,2021-01-01 00:06:54,2021-01-01 00:18:20,2021-01-01 00:38:11,관악구,대학동,광진구,구의제3동,19.85,1191
2,5643,중형승합,2021-01-01 00:21:06,2021-01-01 00:30:51,2021-01-01 01:11:28,서초구,서초1동,구로구,가리봉동,40.62,2437
3,8239,중형승합,2021-01-01 00:26:00,2021-01-01 00:34:10,2021-01-01 00:57:07,성동구,왕십리도선동,강남구,수서동,22.95,1377
4,1303,중형승합,2021-01-01 00:45:00,2021-01-01 00:50:25,2021-01-01 01:12:04,도봉구,쌍문제2동,노원구,상계1동,21.65,1299


## 승차일시 데이터 정제
- 날짜형 데이터로 변환

In [3]:
# '승차일시' -> 날짜형 데이터로 변환
df['승차일시'] = pd.to_datetime(df['승차일시'])

## 승차일시 기준으로 요일 칼럼 생성
- 0: 월요일 ~ 6: 일요일

In [4]:
# 승차일시 기준으로 요일(0~6) 칼럼 생성
df['weekday'] = df['승차일시'].dt.weekday
df.head(2)

,차량고유번호,차량타입,예정일시,배차일시,승차일시,출발지구군,출발지상세,목적지구군,목적지상세,대기시간(m),대기시간(s),weekday
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,강남구,수서동,강남구,개포4동,22.10,1326,4
1,7846,중형 승합,2021-01-01 00:06:54,2021-01-01 00:18:20,2021-01-01 00:38:11,관악구,대학동,광진구,구의제3동,19.85,1191,4


## 승차일시 기준으로 공휴일 칼럼 생성
- 파이썬에서 제공하는 공휴일 패키지들은 정확하지 않음(holiday 패키지, pytimekr 패키지)  
  - from pytimekr import pytimekr
  - import holidays
- 따라서 공공데이터포털, 한국천문연구원이 제공하는 국경일, 공휴일, 기념일 등 조회할 수 있는 openAPI 데이터 활용(대체공휴일도 포함됨, 맨 위에 링크 제시)
- 정제 방법
  1. 공공데이터포털에서 얻은 공휴일 날짜(*locdate*) 정보를 날짜 형(*%Y-%m-%d* 형식)으로 변환
  1. 기존 df의 승차일시에서 날짜 정보만 추출하여 날짜 형으로 변환한 새로운 칼럼(*승차일시(date)*) 생성
  1. *'승차일시(date)'*와 *'locdate'*의 일치 여부를 for문으로 판단 → 공휴일이면 1, 아니면 0

In [5]:
# 공공데이터포털 Open API 데이터 불러오기
from datetime import datetime
import requests
import json
import pandas as pd
from pandas import json_normalize

key = '9dNbZ%2FxAAEdZTV3B%2BxJmwyqgjb4W5TrpKYIYzkFemFeFFseRZdnGbJSSHDf5cJpGZNdMLtIfjnAR%2B%2BBJKGvebg%3D%3D'
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(2021) + '&ServiceKey=' + str(key)
response = requests.get(url)
if response.status_code == 200:
	json_ob = json.loads(response.text)
	holidays_data = json_ob['response']['body']['items']['item']
	holiday_df = json_normalize(holidays_data)
dateName = holiday_df.loc[holiday_df['locdate'] == int(2021), 'dateName']

holiday_df

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20210101,1
1,01,설날,Y,20210211,1
2,01,설날,Y,20210212,1
3,01,설날,Y,20210213,1
4,01,삼일절,Y,20210301,1
5,01,어린이날,Y,20210505,1
6,01,부처님오신날,Y,20210519,1
7,01,현충일,Y,20210606,1
8,01,광복절,Y,20210815,1
9,01,대체공휴일,Y,20210816,1


In [6]:
# locdate 칼럼을 날짜형(%Y-%m-%d)으로 변환
def date_function(x):
    result = str(x)
    return result[0:4] +'-' + result[4:6] + '-' + result[6:8]

holiday_df['locdate'] = pd.to_datetime(holiday_df['locdate'].apply(date_function))
holiday_df

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,2021-01-01,1
1,01,설날,Y,2021-02-11,1
2,01,설날,Y,2021-02-12,1
3,01,설날,Y,2021-02-13,1
4,01,삼일절,Y,2021-03-01,1
5,01,어린이날,Y,2021-05-05,1
6,01,부처님오신날,Y,2021-05-19,1
7,01,현충일,Y,2021-06-06,1
8,01,광복절,Y,2021-08-15,1
9,01,대체공휴일,Y,2021-08-16,1


In [7]:
# 기존 df의 승차일시에서 날짜 정보만 추출하여 날짜 형으로 변환한 새로운 칼럼('승차일시(date)') 생성
df['승차일시(date)'] = pd.to_datetime(df['승차일시'].dt.date, format="%Y-%m-%d")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222765 entries, 0 to 1222764
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   차량고유번호      1222765 non-null  int64         
 1   차량타입        1222765 non-null  object        
 2   예정일시        1222735 non-null  object        
 3   배차일시        1222765 non-null  object        
 4   승차일시        1222765 non-null  datetime64[ns]
 5   출발지구군       1222765 non-null  object        
 6   출발지상세       1222765 non-null  object        
 7   목적지구군       1222765 non-null  object        
 8   목적지상세       1222765 non-null  object        
 9   대기시간(m)     1222765 non-null  float64       
 10  대기시간(s)     1222765 non-null  int64         
 11  weekday     1222765 non-null  int64         
 12  승차일시(date)  1222765 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(7)
memory usage: 121.3+ MB


In [8]:
# 공휴일 여부 판단
holiday_lst = []
for i in range(len(df)):
  if df['승차일시(date)'][i] in holiday_df['locdate'].tolist():
    holiday_lst.append(1)
  else:
    holiday_lst.append(0)
len(holiday_lst) # len(df)와 같은 행 개수(1222765) 출력 확인

df['holiday'] = holiday_lst
df.head(2)

,차량고유번호,차량타입,예정일시,배차일시,승차일시,출발지구군,출발지상세,목적지구군,목적지상세,대기시간(m),대기시간(s),weekday,승차일시(date),holiday
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,강남구,수서동,강남구,개포4동,22.10,1326,4,2021-01-01,1
1,7846,중형 승합,2021-01-01 00:06:54,2021-01-01 00:18:20,2021-01-01 00:38:11,관악구,대학동,광진구,구의제3동,19.85,1191,4,2021-01-01,1


In [ ]:
print('공휴일: {0}개 행, 비공휴일: {1}개 행'.format( len(df.loc[(df['holiday'] == 1)]) , len(df.loc[(df['holiday'] == 0)]) ))

공휴일: 25018개 행, 비공휴일: 1197747개 행


## 공휴일 다음 날 칼럼 생성
- 공휴일 다음 날이면 1, 아니면 0
- holiday_df의 *'locdate'* + **day(1)** == df의 *'승차일시(date)'* 이면 1, 아니면 0

In [9]:
# 공휴일 다음날 날짜 생성(+ day1)
holiday_df['next_locdate'] = holiday_df['locdate'] + timedelta(days=1)

In [10]:
next_holiday_lst = []
for i in range(len(df)):
  if df['승차일시(date)'][i] in holiday_df['next_locdate'].tolist():
    next_holiday_lst.append(1)
  else:
    next_holiday_lst.append(0)

print(len(next_holiday_lst))  # len(df)와 같은 행 개수(1222765) 출력 확인
df['next_holiday'] = next_holiday_lst
df.head(2)

1222765


,차량고유번호,차량타입,예정일시,배차일시,승차일시,출발지구군,출발지상세,목적지구군,목적지상세,대기시간(m),대기시간(s),weekday,승차일시(date),holiday,next_holiday
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,강남구,수서동,강남구,개포4동,22.10,1326,4,2021-01-01,1,0
1,7846,중형 승합,2021-01-01 00:06:54,2021-01-01 00:18:20,2021-01-01 00:38:11,관악구,대학동,광진구,구의제3동,19.85,1191,4,2021-01-01,1,0


In [11]:
print('공휴일 다음 날: {0}개 행, 공휴일 다음 날이 아닌 날: {1}개 행'.format( len(df.loc[(df['next_holiday'] == 1)]) , len(df.loc[(df['next_holiday'] == 0)]) ))

공휴일 다음 날: 49240개 행, 공휴일 다음 날이 아닌 날: 1173525개 행


## 시군구행정동 인코딩 변수 생성

In [85]:
# 출발지 코딩북
import pandas as pd
encoding_df_start = pd.read_excel('/content/drive/MyDrive/03. 장애인 콜택시 논문/데이터/행정경계 데이터/시군구 코딩북.xlsx')
# encoding_df_start[['시도_코딩']] = encoding_df_start[['시도_코딩']].astype(int)
# encoding_df_start[['시도_코딩', '출발지구군_코딩','출발지상세_코딩' ]] = encoding_df_start[['시도_코딩', '출발지구군_코딩','출발지상세_코딩']].astype(str)
encoding_df_start.rename(columns={'시도_코딩':'시도_코딩_x'}, inplace=True)
encoding_df_start.head()

,시도,출발지구군,출발지상세,시도_코딩_x,출발지구군_코딩,출발지상세_코딩
0,서울시,강남구,신사동,1.0,1.0,1
1,서울시,강남구,논현1동,1.0,1.0,2
2,서울시,강남구,논현2동,1.0,1.0,3
3,서울시,강남구,압구정동,1.0,1.0,4
4,서울시,강남구,청담동,1.0,1.0,5


In [86]:
# 목적지 코딩북
encoding_df_end = pd.read_excel('/content/drive/MyDrive/03. 장애인 콜택시 논문/데이터/행정경계 데이터/시군구 코딩북.xlsx')
encoding_df_end.columns = ['시도', '목적지구군', '목적지상세', '시도_코딩', '목적지구군_코딩', '목적지상세_코딩']
# encoding_df_end[['시도_코딩']] = encoding_df_end[['시도_코딩']].astype(int)
# encoding_df_end[['시도_코딩', '목적지구군_코딩', '목적지상세_코딩' ]] = encoding_df_end[['시도_코딩', '목적지구군_코딩', '목적지상세_코딩']].astype(str)
encoding_df_end.head()

,시도,목적지구군,목적지상세,시도_코딩,목적지구군_코딩,목적지상세_코딩
0,서울시,강남구,신사동,1.0,1.0,1
1,서울시,강남구,논현1동,1.0,1.0,2
2,서울시,강남구,논현2동,1.0,1.0,3
3,서울시,강남구,압구정동,1.0,1.0,4
4,서울시,강남구,청담동,1.0,1.0,5


## 출발지구군 및 출발지상세 / 목적지구군 및 목적지상세 中 지역명 수정

In [88]:
def change_local(x):
  if x in ['제기제1동', '제기제2동']:
    return "제기동"
  elif x == '전농제3동':
    return '전농제2동'
  elif x == '답십리제3동':
    return '답십리제1동'
  elif x == '답십리제4동':
    return '답십리제2동'
  elif x == '장안제3동':
    return "장안제1동"
  elif x == '장안제4동':
    return "장안제2동"
  elif x == '명륜3가동':
    return "혜화동"
  elif x in ['신설동', '용두동']:
    return "용신동"  
  elif x == '고촌면':
    return '고촌읍'
  elif x in ['신당제1동', '신당제2동', '신당제3동', '신당제4동', '신당제6동']:
    return "신당동"
  elif x == '이문제3동':
    return '이문제2동'
  elif x in ['중산1동', '중산2동']:
    return "중산동"
  elif x in ['가좌동', '덕이동']:
    return "송산동"
  elif x == '신도동':
    return "삼송동"
  elif x == '교하읍':
    return "교하동"
  elif x == '검단3동':
    return "원당동"
  elif x == '지금동':
    return '다산2동'
  elif x == '양촌면':
    return '양촌읍'
  elif x == '도농동':
    return '다산1동'
  elif x == '김포1동':
    return "김포본동"
  elif x == '김포2동':
    return '장기본동'
  elif x in ['소사본1동', '소사본2동']:
    return '소사본동'
  elif x == '퇴계원면':
    return '퇴계원읍'
  elif x == '이동':
    return '일동'
  elif x == '남구':
    return '미추홀구'
  elif x == '검단4동':
    return '당하동'
  else:
    return x

# + 성남시 중원구 "중동" -> "중앙동"
cond = (df['목적지구군'] == '성남시중원구') & (df['목적지상세'] == '중동')
df.loc[cond, '목적지상세'] = '중앙동'

In [89]:
df['출발지구군_수정'] = df['출발지구군'].apply(change_local)
df['출발지상세_수정'] = df['출발지상세'].apply(change_local)
df['목적지구군_수정'] = df['목적지구군'].apply(change_local)
df['목적지상세_수정'] = df['목적지상세'].apply(change_local)

need_df = df.drop(['출발지구군', '출발지상세', '목적지구군', '목적지상세', '승차일시(date)'], axis=1)
print(need_df.columns)
print(len(need_df))
print(encoding_df_start.columns)

Index(['차량고유번호', '차량타입', '예정일시', '배차일시', '승차일시', '대기시간(m)', '대기시간(s)',
       'weekday', 'holiday', 'next_holiday', '출발지구군_수정', '출발지상세_수정',
       '목적지구군_수정', '목적지상세_수정'],
      dtype='object')
1222765
Index(['시도', '출발지구군', '출발지상세', '시도_코딩_x', '출발지구군_코딩', '출발지상세_코딩'], dtype='object')


In [90]:
encode_df = pd.merge(left = need_df, right = encoding_df_start, how = "left", left_on=['출발지구군_수정', '출발지상세_수정'], right_on = ['출발지구군', '출발지상세'])
final_encode_df = pd.merge(left = encode_df, right = encoding_df_end, how = "left", left_on=['목적지구군_수정', '목적지상세_수정'], right_on = ['목적지구군', '목적지상세'])
final_encode_df.drop(['시도_x'], axis=1, inplace=True)

In [91]:
# 필요 없는 칼럼 삭제
final_encode_df.columns
final_encode_df.drop(['출발지구군', '출발지상세', '시도_y', '목적지구군', '목적지상세'], axis=1, inplace=True)
final_encode_df.head(1)

,차량고유번호,차량타입,예정일시,배차일시,승차일시,대기시간(m),대기시간(s),weekday,holiday,next_holiday,출발지구군_수정,출발지상세_수정,목적지구군_수정,목적지상세_수정,시도_코딩_x,출발지구군_코딩,출발지상세_코딩,시도_코딩,목적지구군_코딩,목적지상세_코딩
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,22.1,1326,4,1,0,강남구,수서동,강남구,개포4동,1.0,1.0,21,1.0,1.0,17


In [92]:
# 데이터 형 변환
final_encode_df[['출발지구군_코딩']] = final_encode_df[['출발지구군_코딩']].astype(int).astype('str')
final_encode_df[['출발지상세_코딩']] = final_encode_df[['출발지상세_코딩']].astype(int).astype('str')
final_encode_df[['목적지구군_코딩']] = final_encode_df[['목적지구군_코딩']].astype(int).astype('str')
final_encode_df[['목적지상세_코딩']] = final_encode_df[['목적지상세_코딩']].astype(int).astype('str')
final_encode_df[['시도_코딩_x']] = final_encode_df[['시도_코딩_x']].astype(int).astype('str')
final_encode_df[['시도_코딩']] = final_encode_df[['시도_코딩']].astype(int).astype('str')

# 문자열 앞에 0 삽입
final_encode_df["출발지구군_코딩"] = final_encode_df["출발지구군_코딩"].str.zfill(2)
final_encode_df["출발지상세_코딩"] = final_encode_df["출발지상세_코딩"].str.zfill(2)
final_encode_df["목적지구군_코딩"] = final_encode_df["목적지구군_코딩"].str.zfill(2)
final_encode_df["목적지상세_코딩"] = final_encode_df["목적지상세_코딩"].str.zfill(2)

# 코딩 값 연결하기
final_encode_df['출발지코딩'] = final_encode_df['시도_코딩_x'] + final_encode_df['출발지구군_코딩'] + final_encode_df['출발지상세_코딩']
final_encode_df['목적지코딩'] = final_encode_df['시도_코딩'] + final_encode_df['목적지구군_코딩'] + final_encode_df['목적지상세_코딩']

# 필요없는 칼럼 제거
final_encode_df.drop(['시도_코딩_x', '출발지구군_코딩', '출발지상세_코딩', '시도_코딩', '목적지구군_코딩', '목적지상세_코딩'], axis=1, inplace=True)

final_encode_df.head()

,차량고유번호,차량타입,예정일시,배차일시,승차일시,대기시간(m),대기시간(s),weekday,holiday,next_holiday,출발지구군_수정,출발지상세_수정,목적지구군_수정,목적지상세_수정,출발지코딩,목적지코딩
0,5725,중형승합,2021-01-01 00:04:45,2021-01-01 00:05:54,2021-01-01 00:28:00,22.10,1326,4,1,0,강남구,수서동,강남구,개포4동,10121,10117
1,7846,중형 승합,2021-01-01 00:06:54,2021-01-01 00:18:20,2021-01-01 00:38:11,19.85,1191,4,1,0,관악구,대학동,광진구,구의제3동,10520,10608
2,5643,중형승합,2021-01-01 00:21:06,2021-01-01 00:30:51,2021-01-01 01:11:28,40.62,2437,4,1,0,서초구,서초1동,구로구,가리봉동,11501,10707
3,8239,중형승합,2021-01-01 00:26:00,2021-01-01 00:34:10,2021-01-01 00:57:07,22.95,1377,4,1,0,성동구,왕십리도선동,강남구,수서동,11602,10121
4,1303,중형승합,2021-01-01 00:45:00,2021-01-01 00:50:25,2021-01-01 01:12:04,21.65,1299,4,1,0,도봉구,쌍문제2동,노원구,상계1동,11002,10912
